In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 영상 경로 설정
video_path= './video1.mp4'
cap = cv2.VideoCapture(video_path)

# 영상 이름
video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]

# frame_move = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) // 3)
# cap.set(cv2.CAP_PROP_POS_FRAMES, frame_move)

while cap.isOpened() :
    
    # 영상 읽기
    ret, img = cap.read()
    
    # 마지막 프레임 확인
    if ret is None:
        break
        
    # 이미지 크기 변경
    img = cv2.resize(img, (img.shape[1] //2, img.shape[0]//2),  interpolation=cv2.INTER_AREA)
    img = img[0 : img.shape[0]*2//3, : ]
    img2 = img.copy()
    img3 = img.copy()
#     white_mask = 
#     red_mask = 
#     white_red_mask =
    
    # 1. 색 공간 변환 ( BGR -> YUV)
    yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    #yuv[yuv[:, :, 2]<170] = 0
    yuv[yuv[:, :, 2]<135] = 0
    yuv[yuv[:, :, 1]<110] = 0
    yuv[yuv[:, :, 0]>230] = 0
    img2 = yuv[ : , : , 2]
    
    # 이진화 
    thr, yuv_red = cv2.threshold(img2, 140, 255, cv2.THRESH_BINARY)
    
    # 모폴로지
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    open_mask = cv2.morphologyEx(yuv_red, cv2.MORPH_CLOSE, k)
    # cv2.imshow('open_mask', open_mask)

    # 2. 캐니 엣지 
    edges = cv2.Canny(img, 100,200)

    # bitwise_and 연산
    res_red = cv2.bitwise_and(open_mask, open_mask, mask =edges)
    cv2.imshow('res_red', res_red)
    

    ## 3. 컨투어 / 허프 원 추출 ##
    contours, hierarchy = cv2.findContours(res_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contour = contours[0]
    epsilon = 0.05 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    
    mask_external = np.zeros(open_mask.shape, open_mask.dtype)  
    
    cv2.drawContours(mask_external, [approx], -1, 255, 2)
    cv2.imshow('mask_external', mask_external)

#     cv2.drawContours(img3, contours, -1, (255, 0, 0), 0)
#     cv2.imshow('img3', img3)
    
    _, _, stats_i, centroids_i = cv2.connectedComponentsWithStats(mask_external)
      
    for idx, centroid_i in enumerate(centroids_i) :
        
        if stats_i[idx][0] == 0 and stats_i[idx][1] == 0 :
            continue
        if np.any(np.isnan(centroid_i)) :
            continue       
    
        x, y, w, h, area = stats_i[idx]
        centerX, centerY = int(centroid_i[0]), int(centroid_i[1])
        
        if area < 1700 and area > 200 and abs(w-h) < 5 :
            try : 
                detected_img = img[y-7 : y+h+7,  x-7 : x+w+7]
                gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
                circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=100, param2=45,minRadius=5, maxRadius=38)
                if circles is not None :
                    circles = np.uint16(np.around(circles))
                    for i in circles[0, :] :
                        print(area)
                        cv2.circle(img, ( centerX, centerY ), 2, (0, 255, 0), 2)
                        cv2.rectangle(img, (x-7, y-7), (x+w+7, y+h+7), (0, 0, 255), 2)
            except : 
                continue
    
    cv2.imshow(video_file, img)
    
    if cv2.waitKey(1) == 27 :
        cap.release()
        cv2.destroyAllWindows()

cap.release()
cv2.destroyAllWindows()